In [42]:
import pandas as pd
import pandas_access as mdb
import uuid

from adept.config import RAW_DATA_DIR

In [47]:
import pandas as pd
import sqlite3

con = sqlite3.connect(RAW_DATA_DIR / "functional_traits.db")

In [50]:
df = pd.read_sql_query("SELECT * from angiosperm_traits", con)

df.columns

Index(['sourceDataset', 'termID', 'hasSyn', 'term', 'category', 'trait1',
       'character1', 'trait2', 'character2', 'trait3', 'character3', 'trait4',
       'character4', 'Comments', 'Traits AND/OR Terms', 'Plants Group'],
      dtype='object')

In [90]:

class Traits():
    """ 
    Read the traits from functional traits database
    """
    
    def __init__(self):
        self._connection = sqlite3.connect(RAW_DATA_DIR / "functional_traits.db")
        self.synonyms = self.get_synonyms()
        
    def _read_sql_query(self, sql):
        return pd.read_sql_query(sql, self._connection)
    
    def get_all_traits(self):
        return pd.concat([
            self.get_angiosperm_traits(),
            self.get_bryophyte_traits(),
            self.get_pteridophyte_traits(),
            self.get_gymnosperms_traits()
        ])
    
    def get_angiosperm_traits(self):
        df = self._select_traits_angiosperm_table()
        return self._traits(df)   
    
    def get_bryophyte_traits(self):
        # We want to select both the traits in the bryophyte table, and the ones in the angiosperm table with plant group = Bryophyte         
        df = pd.concat([
            self._read_sql_query("SELECT * from bryophyte_traits"),
            self._select_traits_angiosperm_table('Bryophyte')
        ])
        return self._traits(df)   
    
    def get_pteridophyte_traits(self):
        # We want to select both the traits in the bryophyte table, and the ones in the angiosperm table with plant group = Bryophyte         
        df = pd.concat([
            self._read_sql_query("SELECT * from pteridophyte_traits"),
            self._select_traits_angiosperm_table('Pteridophyte')
        ])
        return self._traits(df)   
    
    def get_gymnosperms_traits(self):
        """
        Gymnosperms are the same as angiosperms, so AC has just added a few extra gymnosperm-only traits 
        """
        df = pd.concat([
            self._select_traits_angiosperm_table(),
            self._select_traits_angiosperm_table('Gymnosperms')
        ])
        return self._traits(df)
    
    def get_synonyms(self):
        synonyms_df = self._read_sql_query("SELECT DISTINCT term, REPLACE(synonym, '_', '-') as synonym from plant_glossary_synonyms")  
        synonyms_df = synonyms_df.groupby('term')['synonym'].apply(set).reset_index().set_index('term') 
        return synonyms_df
    
    def get_terms(self):         
        # We use term, character and synonyms (exploded) to get trait terms
        df = self.get_all_traits()
        cols = ['character', 'trait']
        return pd.concat(
            [
                df[['term'] + cols],
                df[df.synonym.notnull()].explode('synonym')[['synonym'] + cols].rename(columns={'synonym': 'term'})
            ], ignore_index=True
        ).drop_duplicates()    
    
    def get_unique_terms(self):   
        df = self.get_terms()
        return df.term.unique()
            
    def _traits(self, df):
        df = self._stack_trait_columns(df)        
        df = self._synonymise(df)        
        df = df.replace('_','-', regex=True)
        
        return df
    
    def _select_traits_angiosperm_table(self, plant_group='Angiosperms'):
        sql = f"SELECT * from angiosperm_traits WHERE \"Plants Group\"='{plant_group}' OR \"Plants Group\" IS NULL"
        return self._read_sql_query(sql)
    
    def _stack_trait_columns(self, df):
        df['uuid'] = df.apply(lambda _: uuid.uuid4(), axis=1)
        char_trait_cols = ["character", "trait"]

        # Stack the table so all characterX and traitX columns are
        # combined into character and trait column
        df = pd.wide_to_long(df, char_trait_cols, i='uuid', j="x")

        df = df[['term', 'character', 'trait']]
        df = df.reset_index(drop=True)
        df = df.drop_duplicates()
        # If char and trait are empty, the row didn't have trait2, trait3 etc.,
        df = df.dropna(subset=char_trait_cols) 
        return df
    
    def _synonymise(self, df):
        # Add synonyms column  
        return df.join(self.synonyms, 'term', 'left')
        
        
traits = Traits()

df = traits.get_unique_terms()
# df = df.drop_duplicates()
df

# synonyms = traits.get_synonyms()

 

# df.columns



# df[df.synonym.notna()].head()


# def get_angiosperm_traits():
#     df = pd.read_sql_query("SELECT * from angiosperm_traits WHERE \"Plants Group\"='Angiosperms' OR \"Plants Group\" IS NULL", con)
#     return df

# def _query_angiosperm_traits(plant_group='Angiosperms'):
#     pd.read_sql_query("SELECT * from angiosperm_traits WHERE \"Plants Group\"='Angiosperms' OR \"Plants Group\" IS NULL", con)
    
    
# def get_bryophyte_traits():    
    
    
# df = get_angiosperm_traits()

# df.shape

# df['Plants Group'].unique()







array(['v-form', 'ciliate', 'crenate', ..., 'subaquatic', 'feathery',
       'glomeruliform'], dtype=object)

In [ ]:
class ACCDBTraits():
    """ 
    Read the traits from functional traits CCDB
    """
    
    accdb_file_path = RAW_DATA_DIR / 'Functional Traits_Working_File.accdb'
    plant_groups = ['angiosperm', 'bryophyte', 'pteridophyte']
    plants_group_col = 'Plants Group'      
    
    def __init__(self):
        self._df = self._get_traits_df()
    
    def get_plant_group(self, plant_group):  
        return self._df[self._df[self.plants_group_col] == plant_group]       
        
    def get_unique_terms(self, plant_group = None):  
        df = self.get_terms(plant_group)
        return df.term.unique()
    
    def get_colours(self, plant_group = None):
        df = self.get_terms(plant_group)
        return set(itertools.chain.from_iterable(df[df.trait.str.contains('colour')][['term', 'character']].values))
        
    def get_terms(self, plant_group = None):  
        df = self.get_plant_group(plant_group) if plant_group else self._df        
        # We use term, character and synonyms (exploded) to get trait terms
        cols = ['character', 'trait', self.plants_group_col]
        return pd.concat(
            [
                df[['term'] + cols],
                df[df.synonym.notnull()].explode('synonym')[['synonym'] + cols].rename(columns={'synonym': 'term'})
            ], ignore_index=True
        ).drop_duplicates()
                        
    def _get_traits_df(self):
        
        synonyms_df = self._get_synonyms_df()
      
        dfs = [self._get_plant_group_df(plant_group) for plant_group in self.plant_groups]                
        df = pd.concat(dfs, ignore_index=True)
        
        # gymnosperms are the same as angiosperms, so AC has just added a few extra gymnosperm-only traits 
        # Copy the angiosperm traits, and set plant group to gymnosperm
        gymnosperm_df = df[df[self.plants_group_col] == 'angiosperms'].copy()
        gymnosperm_df[self.plants_group_col] = 'gymnosperm'  
        
        # df.append(gymnosperm_df, ignore_index=True)
        df = pd.concat([df, gymnosperm_df])
        
        
        df = df.rename(columns={"Trait 4": "trait4"}) 
                
        df['uuid'] = df.apply(lambda _: uuid.uuid4(), axis=1)
        
        char_trait_cols = ["character", "trait"]
        
        # Stack the table so all characterX and traitX columns are
        # combined into character and trait column
        df = pd.wide_to_long(df, char_trait_cols, i='uuid', j="x")
        
        df = df[['termID', 'term', 'character', 'trait', self.plants_group_col]]
        
        synonyms_df.synonym = synonyms_df.synonym.replace('_','-', regex=True)
        synonyms_df = synonyms_df.groupby('term')['synonym'].apply(set).reset_index().set_index('term')        
        df = df.join(synonyms_df, 'term', 'left')        
        
        # If char and trait are empty, the row probably didn't have trait2, trait3 etc., set
        df = df.dropna(subset=char_trait_cols)                
        
        term_cols = ['term', 'character', 'trait']
        df[term_cols] = df[term_cols].replace('_','-', regex=True)
        df[term_cols] = df[term_cols].applymap(self._normalise_text)
                
        # missing = pd.DataFrame([
        #     {'term': 'stem erect', 'character': 'stem erect', 'trait': 'habit', 'Plants Group': 'angiosperm'
        # }])
        
        excel_terms = self._get_excel_terms_df()
        df = df.append(excel_terms, ignore_index=True)

        return df       
        
    
    def _get_synonyms_df(self):
        df = self._accdb_read_table('plant_glossary_synonyms')
        # We only want term and synonym
        df = df[['term', 'synonym']].drop_duplicates()        
        return df
    
   
                
    def _get_plant_group_df(self, plant_group):
        table_name = f'{plant_group}_traits'
        
        
        
        df = self._accdb_read_table(table_name)
        df[self.plants_group_col] = plant_group        
        
        return df